In [1]:
from pathlib import Path
import shutil
test = [1,2,3,4,5,6]

In [37]:
# cutoutdir = Path("/home/weedsci/matt/SemiF-AnnotationPipeline/data/semifield-developed-images")
cutoutdir = Path("/media/hdd1/mkutu/wir/semifield-developed-images")
glob = sorted(list(cutoutdir.glob("*")))
lastimgs_list = []
for x in glob:
    if not str(x.name).startswith("."):
        lastimgs = sorted(list(Path(cutoutdir, x.name, "images").glob("*.jpg")))[-3:]
        for img in lastimgs:
            lastimgs_list.append(img)

In [38]:
len(lastimgs_list)
savedir = Path("/home/weedsci/matt/SemiF-SyntheticPipeline/data/semifield-synth/possible_backgrounds")
for img in lastimgs_list:
    imgname = Path(img).name
    dst = Path(savedir, imgname)
    # shutil.copy2(img, dst)

1.0